In [1]:
import torch
import MiniFL as mfl

In [2]:
def run_client(i:int, client: mfl.algorithms.gd.Client):
    for _ in range(100):
        loss = client.send_grad_get_loss()
        # print(f"Client {i}: {loss}")
        client.apply_global_step()
        
def run_master(master: mfl.algorithms.gd.Master):
    for i in range(100):
        print(f"Master: {master.round()}")

In [3]:
from sklearn.datasets import load_svmlight_file

NUM_CLIENTS = 20

data, labels = load_svmlight_file("phishing.txt")
enc_labels = labels.copy()
data_dense = data.todense()

eval_data = (torch.from_numpy(data_dense).to(torch.float32), torch.from_numpy(enc_labels).to(torch.float32)[:, None])
clients_data = [(x, y) for x, y in zip(torch.split(eval_data[0], len(eval_data[0]) // NUM_CLIENTS, dim=0), torch.split(eval_data[1], len(eval_data[1]) // NUM_CLIENTS, dim=0))]

model = torch.nn.Linear(eval_data[0].shape[1], 1, bias=False)
loss_fn = torch.nn.BCEWithLogitsLoss()

master, clients = mfl.algorithms.gd.get_master_and_clients(
    lr=200,
    clients_data=clients_data,
    eval_data=eval_data,
    model=model,
    loss_fn=loss_fn,
)

In [4]:
import threading

client_threads = []
for i, client in enumerate(clients):
    client_threads.append(threading.Thread(target=run_client, args=(i, client)))
    client_threads[-1].start()
    
master_thread = threading.Thread(target=run_master, args=(master,))
master_thread.start()

master_thread.join()
for t in client_threads:
    t.join()
    


Master: 0.6956520676612854
Master: 0.6956655979156494
Master: 0.6947044730186462
Master: 0.6927826404571533
Master: 0.6928551197052002
Master: 0.6928449273109436
Master: 0.6969215869903564
Master: 0.6982445120811462
Master: 0.6982307434082031
Master: 0.6966410279273987
Master: 0.697031557559967
Master: 0.6968624591827393
Master: 0.6968624591827393
Master: 0.6969335079193115
Master: 0.6969401240348816
Master: 0.6976301074028015
Master: 0.6975311040878296
Master: 0.6976613402366638
Master: 0.6913421750068665
Master: 0.6915144324302673
Master: 0.6910050511360168
Master: 0.6899986267089844
Master: 0.6910388469696045
Master: 0.6910390853881836
Master: 0.691039502620697
Master: 0.6909781098365784
Master: 0.6795223355293274
Master: 0.6799498200416565
Master: 0.677959680557251
Master: 0.6779078841209412
Master: 0.6780370473861694
Master: 0.6772381663322449
Master: 0.677409827709198
Master: 0.676660418510437
Master: 0.6774293184280396
Master: 0.677143394947052
Master: 0.6761600971221924
Master:

In [6]:
clients[0].data_sender.n_bits_passed

3200